In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier  # Or import xgboost as xgb
from sklearn.metrics import precision_recall_auc_score, f1_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# Load (example for fraud_data)
df = pd.read_parquet('../data/processed/fraud_engineered.parquet')
X = df.drop('class', axis=1)  # Features
y = df['class']  # Target

# Stratified split (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f"Train fraud rate: {y_train.mean():.2%}, Test: {y_test.mean():.2%}")

ModuleNotFoundError: No module named 'sklearn'